In [2]:
import numpy as np
from numpy.random import normal
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from pandas import Series, DataFrame
from scipy.stats import norm
from numpy.random import normal, multivariate_normal

# EMアルゴリズムの概要
* 教師なし学習によるクラスタリングのアルゴリズム
* 確率モデルのパラメータを最尤推定する手法の一つ
* 尤度の期待値を最大化するようなパラメータを求めるためにk平均法のように反復して計算する

# EMアルゴリズムを用いて今回やること

手書き文字を分類する。以下のステップで実行される

1. 手書きサンプルから代表文字の生成 (ベルヌーイ分布を用いた最尤推定法)
2. 複数の文字が混在した手書き文字サンプルの分類(混合ベルヌ−イ分布を用いた最尤推定法 <- EMアルゴリズムを使う)

# 1. 手書きサンプルから代表文字の生成

## 概略

1. 平均化による代表文字の生成
2. ベルヌーイ分布を用いた最尤推定法による代表文字の生成
3. 1と２は結果的に同じになる

## 平均化による代表文字の生成
各手書き画像(トレーニングセット)のピクセルを横一列に並べて色(黒白)を1,0で表したベクトルを用意する
 (文字データはモノクロ2段調)
n番目の画像に対応するベクトルをXnとすると第i成分 [Xn]i を見るとi番目のピクセルの色が分かる。これらの平均を取れば、特定の数字の各i成分目を平均化したベクトルμを用意できる。

$$ \mu = \frac{1}{N} \Sigma_{n=1}^{N}x_n $$  (7.1)

## ベルヌーイ分布を用いた最尤推定法による代表文字の生成
ランダムに手書き文字を生成する**画像生成器** があるとする。これは、下記のような濃淡のあるような文字を生成するとする。

[![https://gyazo.com/c8b7471de8a3cc39fc9d9a714c33e6e4](https://i.gyazo.com/c8b7471de8a3cc39fc9d9a714c33e6e4.png)](https://gyazo.com/c8b7471de8a3cc39fc9d9a714c33e6e4)

* 上記の画像のピクセルの濃淡を0~1で表わし(ベクトルμ)、「生成される画像のi番目のピクセルを黒にする」確立として考えると、ランダムな画像を生成することができる。この時、ベクトルμを**画像生成器**とを生成することができる。
* では、どのような画像生成器を選べばいいのか？　
* それはトレーニングセット（内の特定の数字のデータ）と、画像生成器が生成する数字が同じになる確率が最大化する画像生成器を選出する。
* これをするのが**ベルヌーイ分布**になる

* ある特定の画像データxのi番目(1~D)のピクセルの色(0 or 1)が得られる確立をp_iとすると、p_iは下記の条件が成り立つ必要がある。
$$ i番目のピクセルが黒の場合(x_i = 1): p_i = \mu_i$$
$$ i番目のピクセルが黒の場合(x_i = 0): p_i = 1 - \mu_i$$

* 上記の条件は下記の式で表すことができる (**ベルヌーイ分布**)

$$ p_i = \mu_{i} ^{x_i}(1-\mu_i)^{1-x_i} $$ 

* 従って、全ピクセルについて同じ色に成る確立は以下のとおり

$$ p(x) = \Pi_{i=1}^{D}p_i = \Pi_{i=1}^{D}\mu_{i}^{x_i}(1-\mu_i)^{1-x_i} $$

* 更に特定の数字の全トレーニングせっとに一致する画像が得られる確立配下の通り (**尤度関数**)

$$ P = \Pi_{n=1}^{N} p(x_n)  = \Pi_{n=1}^{N}  \Pi_{i=1}^{D} \mu_{i}^{{[x_n]}_i} (1-\mu_i) ^ {1-{[x_n]}_i}$$

* 上記のPが最大化するμを求めるとそれが、(7.1)と等しくなる。

## 2. 複数の文字が混在した手書き文字サンプルの分類

### 概略
* 複数の数字の手書き画像データがある場合に、文字の種類ごとに画像データを分類する方法を求める
* 混合ベルヌーイ分布を用いて最尤推定法を実施する
 * ここでEMアルゴリズムが必要になる

### 尤度関数(モデル)の生成
* 今回は複数の数字の手書き画像データがある。つまり全部でK種類の数字を含むセットがある。
* 各数字ごとに画像生成器 u_kがあるとなる。
* 各画像生成器から元の数字画像が得られる確立は以下のとおり。

$$ p_{\mu_k}(x) = \Pi_{i=1}^{D}{[\mu_k]}_{i}^{x_i}(1-{[\mu_k]}_i)^{1-x_i} $$ (7.10)

* k番目の画像生成器を選ぶ確立をpi_kとすると、kが１〜Kの時のpi_kの合計は　１になる。
* すべての画像生成器を使って、特定の画像xが得られる確立は次の通り

$$ p(x) = \Sigma^{K}_{k=1} \pi_k p_{\mu_k} (x) $$ (7.12)

* よって、K種類の数字を含む全トレーニングセットと、選出された各画像生成器が生成する画像が一致する確立は以下の通り。
    *  全これが **尤度関数** になる

$$ P = \Pi^N_{n=1}p(x_n) = \Pi^N_{n=1}\Sigma^K_{k=1}\pi_k p_{\mu_k}(x_n)$$ (7.13)

### EMアルゴリズムを利用してモデルを最適化するパラメタを用いる
* (7.13)のような積の計算(Π)と和の計算(∑)が混在している尤度関数を最大にするパラメタを用いるアルゴリズム = EMアルゴリズム

* ステップ1: K個の画像生成器(μ_k)を準備する。又、各画像生成器を選択する確立(π_k)も準備。

* ステップ2: 全画像生成器から、x_n画像を得られる確立は(7.12)の通りだが、特定のk番目の画像生成器から、x_nを得られる可能性は以下のとおり
    * これはx_nがそれぞれの画像生成器に所属する確立

$$ \gamma_{nk} = \frac{\pi_k p_{\mu_k} (x_n)}{ \Sigma^{K}_{k' = 1} \pi_{k'} p_{\mu_{k'}}(x_n) }$$

* ステップ3: 上記に基いて、新しい画像生成器を作り直し、又、それぞれの画像生成器を選択する確立を計算しなおす(更新する)

$$ \mu_k = \frac{\Sigma_{n=1}^N \gamma_{nk} x_n}{ \Sigma_{n=1}^N \gamma_{nk}} をk = (1~K)で実行$$
$$ \pi_k = \frac{\Sigma_{n=1}^N \gamma_{nk}}{N}をk = (1~K)で実行$$

* ステップ4: ステップ４で求めた新しい値を用いて、ステップ2に戻る
* ステップ5: 上記イテレーションで尤度関数が最大化する

### 注意点
* ステップ1で選択したk=1~Kのμ_k とπ_kによって結果が変わる可能性もある。

# 手書き文字1~10を分類するケース

* 画像生成器の作成
[![https://gyazo.com/8a3aa603d3b51f96cbb8186f25c737c2](https://i.gyazo.com/8a3aa603d3b51f96cbb8186f25c737c2.png)](https://gyazo.com/8a3aa603d3b51f96cbb8186f25c737c2)

* 最終的な画像生成器を用いた分類結果
[![https://gyazo.com/51333c500376bfa6d105545f0481e427](https://i.gyazo.com/51333c500376bfa6d105545f0481e427.png)](https://gyazo.com/51333c500376bfa6d105545f0481e427)



* 見ての通り余り正確ではない
* 例えば1